<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Running-ALE" data-toc-modified-id="Running-ALE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Running ALE</a></span><ul class="toc-item"><li><span><a href="#Finding-failed-jobs" data-toc-modified-id="Finding-failed-jobs-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Finding failed jobs</a></span></li><li><span><a href="#ALE-on-failed-jobs" data-toc-modified-id="ALE-on-failed-jobs-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>ALE on failed jobs</a></span></li><li><span><a href="#Finding-failed-jobs" data-toc-modified-id="Finding-failed-jobs-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Finding failed jobs</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Notes" data-toc-modified-id="Notes-5.3.0.1"><span class="toc-item-num">5.3.0.1&nbsp;&nbsp;</span>Notes</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Summarizing-results" data-toc-modified-id="Summarizing-results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Summarizing results</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* run ALE on AlmeidaA-2019 MAGs (those selected for analysis with DeepMAsED)

# Var

In [20]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE/'

# file of MAGs used for DeepMAsED analysis 
MAGs_n143_file = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019//samples_Almeida-Pasolli.txt'

# bam file dir (mapping of metagenome reads to corresponding MAGs)
map_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map/'

# ALE executable
ALE_exe = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/ALE/src/ALE'

# params
conda_env = 'base'
threads = 24

# Init

In [21]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(doParallel)

library(future)
library(future.batchtools)
library(future.apply)
options(future.wait.interval = 2.0)

# Load

In [31]:
MAGs_n143 = read.delim(MAGs_n143_file, sep='\t') %>%
    mutate(Taxon = gsub('[-.]', '_', Taxon))

MAGs_n143 %>% nrow %>% print
MAGs_n143$Taxon %>% unique %>% length %>% print
MAGs_n143 %>% head(n=3)

[1] 1090
[1] 1090


Taxon,Fasta,Sample,Read1,Read2
VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1024/VogtmannE_2016__MMRS65205033ST-27-0-0__bin.43.fa.gz,ERR1293531,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R2_final.fq.gz
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1408/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.46.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1472/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.34.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz


In [32]:
# adding bam files
bam_files = list.files(map_dir, '*.bam$', recursive=TRUE, full.names=TRUE)
bam_files = data.frame(Taxon = bam_files %>% dirname %>% basename,
                       Bam = bam_files)

bam_files %>% nrow %>% print
bam_files %>% head(n=3)

[1] 1090


Taxon,Bam
BackhedF_2015__SID201_4M__bin_1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//BackhedF_2015__SID201_4M__bin_1/ERR525761.bam
BackhedF_2015__SID201_4M__bin_10,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//BackhedF_2015__SID201_4M__bin_10/ERR525761.bam
BackhedF_2015__SID201_4M__bin_12,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//BackhedF_2015__SID201_4M__bin_12/ERR525761.bam


In [33]:
setdiff(MAGs_n143$Taxon, bam_files$Taxon) %>% length
setdiff(bam_files$Taxon, MAGs_n143$Taxon) %>% length

[1] 0

[1] 0

In [34]:
# joining
intersect(MAGs_n143$Taxon, bam_files$Taxon) %>% length %>% print

MAGs_n143 = MAGs_n143 %>%
    inner_join(bam_files, c('Taxon'))

MAGs_n143 %>% nrow %>% print
MAGs_n143 %>% head(n=3)

[1] 1090


Warning message:
“Column `Taxon` joining character vector and factor, coercing into character vector”

[1] 1090


Taxon,Fasta,Sample,Read1,Read2,Bam
VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1024/VogtmannE_2016__MMRS65205033ST-27-0-0__bin.43.fa.gz,ERR1293531,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43/ERR1293531.bam
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1408/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.46.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46/ERR1293612.bam
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1472/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.34.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34/ERR1293612.bam


In [35]:
# ALE output file name
MAGs_n143 = MAGs_n143 %>%
    mutate(ALE = sapply(Taxon, function(x) file.path(work_dir, paste0(x, '.txt'))))
                        
MAGs_n143 %>% head(n=3)

Taxon,Fasta,Sample,Read1,Read2,Bam,ALE
VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1024/VogtmannE_2016__MMRS65205033ST-27-0-0__bin.43.fa.gz,ERR1293531,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293531/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43/ERR1293531.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43.txt
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1408/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.46.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46/ERR1293612.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_46.txt
VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1472/VogtmannE_2016__MMRS51737257ST-27-0-0__bin.34.fa.gz,ERR1293612,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR1293612/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34/ERR1293612.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS51737257ST_27_0_0__bin_34.txt


# Running ALE

In [36]:
# cluster resources
resources = list(h_rt = '00:59:00',
                 h_vmem = '8G',
                 threads = '1',
                 conda.env = 'py3_physeq')     # conda env with batchtools installed
plan(batchtools_sge, resources=resources, workers=40)

In [37]:
ALE_params = function(x, exe){
    cmd = '{exe} {bam} {fasta} {outfile}'
    cmd = glue::glue(cmd, exe=exe, bam=x[['Bam']], fasta=x[['Fasta']], outfile=x[['ALE']])
    return(cmd)
}

ALE_cmds = apply(MAGs_n143, 1, ALE_params, exe=ALE_exe) %>%
    as.list

ALE_cmds %>% length %>% print
ALE_cmds[[1]]

[1] 1090


[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/ALE/src/ALE /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43/ERR1293531.bam /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1024/VogtmannE_2016__MMRS65205033ST-27-0-0__bin.43.fa.gz /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43.txt"

In [38]:
bash_job = function(cmd, conda_env, stdout=TRUE, stderr=FALSE){
    # cmd : string; commandline job (eg., 'ls -thlc')
    # conda_env : string; conda environment name
    cmd = sprintf('. ~/.bashrc; conda activate %s; %s', conda_env, cmd)
    cmd = sprintf('-c "%s"', cmd)
    system2('bash', cmd, stdout=stdout, stderr=stderr)
}

In [39]:
# apply function on cluster
job_ret = future_lapply(ALE_cmds, FUN = function(x) bash_job(x, conda_env=conda_env))
job_ret %>% length %>% print
job_ret[[1]]

[1] 1090


[1] "BAM file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43/ERR1293531.bam"             
 [2] "Assembly fasta file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1024/VogtmannE_2016__MMRS65205033ST-27-0-0__bin.43.fa.gz"  
 [3] "ALE Output file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43.txt"                 
 [4] "Reading in assembly..."                                                                                                                                                                       
 [5] "Reading in the map and computing statistics..."                                                                                                                                               
 [6] "Insert length and std not given, will be calculated from input map."                                                                                                                          
 [7] "Found FR sample avg insert length to be 307.284776 from 9892 mapped reads"                                                                                                                    
 [8] "Found FR sample insert length std to be 59.733316"                                                                                                                                            
 [9] "There were 10683 total reads, 10683 paired (10298 properly mated), 281 proper singles, 104 improper reads (60 chimeric). (0 reads were unmapped)"                                             
[10] "Saved library parameters to /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//VogtmannE_2016__MMRS65205033ST_27_0_0__bin_43.txt.param"
[11] "Computing read placements and depths"                                                                                                                                                         
[12] "FR orientation with 10298 reads, 10298 mapped, 0 unmapped, 10298 placed, 0 mappedButNotPlaced"                                                                                                
[13] "RF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                                                            
[14] "FF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                                                            
[15] "NOT_PROPER_FR orientation with 44 reads, 44 mapped, 0 unmapped, 44 placed, 0 mappedButNotPlaced"                                                                                              
[16] "NOT_PROPER_RF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                                                 
[17] "NOT_PROPER_FF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                                                 
[18] "UNMAPPED_PAIR orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                                                 
[19] "CHIMER orientation with 60 reads, 60 mapped, 0 unmapped, 60 placed, 0 mappedButNotPlaced"                                                                                                     
[20] "SINGLE_UNMAPPED_MATE orientation with 281 reads, 281 mapped, 0 unmapped, 281 placed, 0 mappedButNotPlaced"                                                                                    
[21] "UNMAPPED_MATE orientation with 0 reads, 0 mapped, 0 unm

## Finding failed jobs

In [40]:
P = map_dir %>% dirname %>% file.path('ALE')
feat_files = list.files(P, '*.txt$', recursive=TRUE) %>%
    gsub('\\.txt$', '', .)
feat_files %>% length
feat_files %>% head(n=3)

[1] 984

[1] "BackhedF_2015__SID201_4M__bin_1"  "BackhedF_2015__SID201_4M__bin_10"
[3] "BackhedF_2015__SID201_4M__bin_12"

In [41]:
failed_jobs = MAGs_n143 %>%
    filter(! Taxon %in% feat_files)

failed_jobs %>% nrow %>% print
failed_jobs %>% head(n=3)

[1] 106


Taxon,Fasta,Sample,Read1,Read2,Bam,ALE
NielsenHB_2014__MH0032__bin_6,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1626/NielsenHB_2014__MH0032__bin.6.fa.gz,ERR209130,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209130/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209130/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__MH0032__bin_6/ERR209130.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__MH0032__bin_6.txt
NielsenHB_2014__MH0107__bin_3,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1684/NielsenHB_2014__MH0107__bin.3.fa.gz,ERR209342,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209342/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209342/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__MH0107__bin_3/ERR209342.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__MH0107__bin_3.txt
NielsenHB_2014__O2_UC47_2__bin_42,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1699/NielsenHB_2014__O2_UC47_2__bin.42.fa.gz,ERR209633,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209633/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209633/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__O2_UC47_2__bin_42/ERR209633.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__O2_UC47_2__bin_42.txt


## ALE on failed jobs

In [42]:
# cluster resources
resources = list(h_rt = '00:59:00',
                 h_vmem = '64G',
                 threads = '1',
                 conda.env = 'py3_physeq')     # conda env with batchtools installed
plan(batchtools_sge, resources=resources, workers=40)

In [43]:
ALE_params = function(x, exe){
    cmd = '{exe} {bam} {fasta} {outfile}'
    cmd = glue::glue(cmd, exe=exe, bam=x[['Bam']], fasta=x[['Fasta']], outfile=x[['ALE']])
    return(cmd)
}

ALE_cmds = apply(failed_jobs, 1, ALE_params, exe=ALE_exe) %>%
    as.list

ALE_cmds %>% length %>% print
ALE_cmds[[1]]

[1] 106


[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/ALE/src/ALE /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__MH0032__bin_6/ERR209130.bam /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1626/NielsenHB_2014__MH0032__bin.6.fa.gz /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__MH0032__bin_6.txt"

In [44]:
# apply function on cluster
job_ret = future_lapply(ALE_cmds, FUN = function(x) bash_job(x, conda_env=conda_env))
job_ret %>% length %>% print
job_ret[[1]]

[1] 106


character(0)
attr(,"status")
[1] 136

## Finding failed jobs

In [45]:
P = map_dir %>% dirname %>% file.path('ALE')
feat_files = list.files(P, '*.txt$', recursive=TRUE) %>%
    gsub('\\.txt$', '', .)
feat_files %>% length
feat_files %>% head(n=3)

[1] 984

[1] "BackhedF_2015__SID201_4M__bin_1"  "BackhedF_2015__SID201_4M__bin_10"
[3] "BackhedF_2015__SID201_4M__bin_12"

In [46]:
failed_jobs = MAGs_n143 %>%
    filter(! Taxon %in% feat_files)

failed_jobs %>% nrow %>% print
failed_jobs %>% head(n=3)

[1] 106


Taxon,Fasta,Sample,Read1,Read2,Bam,ALE
NielsenHB_2014__MH0032__bin_6,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1626/NielsenHB_2014__MH0032__bin.6.fa.gz,ERR209130,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209130/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209130/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__MH0032__bin_6/ERR209130.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__MH0032__bin_6.txt
NielsenHB_2014__MH0107__bin_3,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1684/NielsenHB_2014__MH0107__bin.3.fa.gz,ERR209342,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209342/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209342/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__MH0107__bin_3/ERR209342.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__MH0107__bin_3.txt
NielsenHB_2014__O2_UC47_2__bin_42,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/all_MAGs/SGB_genome_fastas_part1/1699/NielsenHB_2014__O2_UC47_2__bin.42.fa.gz,ERR209633,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209633/R1_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/ERR209633/R2_final.fq.gz,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/map//NielsenHB_2014__O2_UC47_2__bin_42/ERR209633.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//NielsenHB_2014__O2_UC47_2__bin_42.txt


#### Notes

* It appears that the failed MAGs are a result of lack of coverage

# Summarizing results

In [47]:
ALE_res_files = list.files(work_dir, '*.txt$', recursive=TRUE, full.names=TRUE)
ALE_res_files %>% length %>% print
ALE_res_files %>% head

[1] 984


[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_1.txt" 
[2] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_10.txt"
[3] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_12.txt"
[4] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_16.txt"
[5] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_5.txt" 
[6] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/PasolliE-2019/samples_Almeida-Pasolli/ALE//BackhedF_2015__SID201_4M__bin_6.txt"

# sessionInfo

In [48]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.2 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] bindrcpp_0.2.2          future.apply_1.0.0      future.batchtools_0.7.1
 [4] future_1.9.0            doParallel_1.0.11       iterators_1.0.10       
 [7] foreach_1.4.4           dat